# 이미지 분류 예제(Gradio 예제)

In [1]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
from io import BytesIO

#TensorFlow MobileNetV2 모델 로드
model=tf.keras.applications.MobileNetV2(weights="imagenet");

d:\241223\aiProject2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [2]:
def predict_image(image_url):
    try:
        #URL에서 이미지 가져오기
        response=requests.get(image_url);
        image=Image.open(BytesIO(response.content)).resize((224,224));  #BytesIO 사용하여 이미지 열기
        
        #이미지를 배열로 변환
        image_array=tf.keras.preprocessing.image.img_to_array(image);
        image_array=tf.expand_dims(image_array, axis=0)     #배치 차원 추가
        image_array=tf.keras.applications.mobilenet_v2.preprocess_input(image_array);   #전처리
        
        #예측 수행
        predictions=model.predict(image_array);
        decoded_predictions=tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=3)[0];   #상위 3개 예측결과 반환
        
        #Gradio Label 컴포넌트에 맞게 결과 형식 변경
        #{label: confidence1, label2: confidence2, ...} 형식으로 반환
        result={label: float(prob) for (_, label, prob) in decoded_predictions}
        return result;
    except Exception as e:
        return {"error": 1.0}   #에러 발생 시 기본갑 반환

In [3]:
#Gradio 인터페이스 생성
iface=gr.Interface(
    fn=predict_image,
    inputs=gr.Textbox(label="이미지 URL 입력"),
    outputs=gr.Label(num_top_classes=3, label="예측결과"),
    title="음식 이미지 분류",
    description="이미지 URL을 입력하면 상위 3개의 예측 결과를 확률과 함께 표시합니다."
)

#인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True);
# 예시 이미지 URL : https://health.chosun.com/site/data/img_dir/2024/04/19/2024041901914_0.jpg



* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Keyboard interruption in main thread... closing server.


In [ ]:
#자원 반환
iface.close();

# 이미지 분류 예제(Gradio+gemma2 사용)

In [1]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
from io import BytesIO
from langchain_community.chat_models import ChatOllama

#TensorFlow MobileNetV2 모델 로드
model=tf.keras.applications.MobileNetV2(weights="imagenet");

OLLAMA_SERVER="http://localhost:11434"; #로컬 서버주소
MODEL_NAME="gemma2"     #사용하려는 Ollama 모델 이름

#Ollama를 사용해 음식 설명 생성
def get_food_description_with_langchain(food_name):
    """
    LangChain ChatOllama를 사용하여 음식 설명 생성
    """
    try:
        chat=ChatOllama(base_url=OLLAMA_SERVER, model=MODEL_NAME);
        prompt=f"Tell me about {food_name}.";
        response=chat.predict(prompt);
        return response;
    except Exception as e:
        return f"Failed to retrieve description: {e}";

d:\241223\aiProject2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#이미지 예측 함수
def predict_image_with_description(image_url):
    """
    이미지 URL을 받아 음식 예측과 Ollama 설명을 반환
    """
    try:
        #URL에서 이미지 가져오기
        response=requests.get(image_url);
        image=Image.open(BytesIO(response.content)).resize((224,224))   #BytesIO 사용하여 이미지 열기
        
        #이미지를 배열로 변환
        image_array=tf.keras.preprocessing.image.img_to_array(image);
        image_array=tf.expand_dims(image_array, axis=0) #배치차원 추가
        image_array=tf.keras.applications.mobilenet_v2.preprocess_input(image_array);   #전처리
        
        #예측 수행
        predictions=model.predict(image_array);
        decoded_predictions=tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=3)[0];   #상위 3개 예측 결과 반환
        
        #예측결과 형식화
        result={label: float(prob) for (_, label, prob) in decoded_predictions}
        
        #가장 높은 확률의 예측값으로 Ollama 설명 생성
        top_food=decoded_predictions[0][1];     #가장 확률이 높은 음식이름
        description=get_food_description_with_langchain(top_food);
        
        return result, description;   #예측 결과와 Ollama 설명 반환
    except Exception as e:
        return {"error": 1.0}, f"Error: {e}"    #에러 발생 시 기본값 반환
    

In [ ]:
#Gradio 인터페이스 생성
iface=gr.Interface(
    fn=predict_image_with_description,
    inputs=gr.Textbox(label="이미지 URL 입력"),
    outputs=[
        gr.Label(num_top_classes=3, label="예측결과"),      #상위 3개 예측결과
        gr.Textbox(label="음식 설명", interactive=False)    #Ollama로 생성한 설명 출력
    ],
    title="음식 이미지 분류 및 설명 생성기",
    description="이미지 URL을 입력하면 음식 분류 결과와 설명을 제공합니다."
);

#인터페이스 실행
iface.launch(server_port=7863, server_name="0.0.0.0", debug=True, share=True);


* Running on local URL:  http://0.0.0.0:7863
* Running on public URL: https://21b9195ef9fed82e86.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


C:\Users\human-10\AppData\Local\Temp\ipykernel_13324\1703203776.py:21: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  chat=ChatOllama(base_url=OLLAMA_SERVER, model=MODEL_NAME);
C:\Users\human-10\AppData\Local\Temp\ipykernel_13324\1703203776.py:23: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=chat.predict(prompt);


In [11]:
#자원 반환
iface.close();